In [92]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from toshl_fixer.common import DATA_DIR, csv_name

In [93]:
def vectorize_desc(data):
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform(data)
    feature_names = vectorizer.get_feature_names()
    dense = vectors.todense()
    denselist = dense.tolist()
    return pd.DataFrame(denselist, columns=feature_names)

def create_vector(data):
    df = vectorize_desc(data['desc'])
    df['amount'] = data['amount'] 
    return df


In [94]:
training_data = pd.read_csv(DATA_DIR / csv_name(from_date="2019-12-01", to_date="2019-12-31"))
test_data = pd.read_csv(DATA_DIR / csv_name(from_date="2019-11-01", to_date="2019-11-31"))
train_len = len(training_data)
data = pd.concat([test_data, training_data], ignore_index=True, sort=False)
data = data.fillna('missing')
data['cat/tag'] = data.category.str.cat(data.tag, sep='/')
del data['id']
del data['category']	
del data['tag']

In [95]:
vec = create_vector(data)
training_vec = vec.iloc[:train_len]
training_data = data.iloc[:train_len]
test_vec = vec.iloc[train_len:]
test_data = data.iloc[train_len:]
test_data

,desc,amount,cat/tag
91,סופרפארם מגדלי אלון,-67.90,Other/missing
92,רולדין יהודה המכבי,-10.00,Other/missing
93,"טיב טעם יהודה המכבי ת""א",-41.21,Basics/groceries
94,קיט קט,-57.40,Basics/missing
95,תן ביס,-35.75,unsorted/missing
...,...,...,...
266,APPLE.COM/BILL ITUNES.COM IE,-3.90,Leisure/devices
267,רובין הוד,-20.00,Leisure/apps
268,טיב טעם יהודה,-110.33,Basics/groceries
269,קיוסק יהודה מכ,-32.00,Leisure/Eating out


In [96]:
from sklearn import tree
X = training_vec
Y = training_data['cat/tag']
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, Y)

In [97]:
for index, row in test_vec.iterrows():
    test_data.at[index, 'label'] = clf.predict(row.values.reshape(1, -1))[0]
 
test_data
	
test_data[test_data['label'] != test_data['cat/tag']]
# test_vec.iloc[1].values

/home/noam/.pyenv/versions/toshl-fixer/lib/python3.8/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/noam/.pyenv/versions/toshl-fixer/lib/python3.8/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,desc,amount,cat/tag,label
109,GETT,-51.70,Transport/missing,Other/missing
129,NEXT DIRECTORY INTERNET,-177.00,Transfer/missing,Other/missing
133,GETT,-40.49,Transport/missing,Other/missing
134,GETT,-57.97,Transport/missing,Other/missing
137,"חניון רכב די.סי בע""מ",-22.00,Other/missing,Health & Personal Care/missing
...,...,...,...,...
258,כימרה,-131.00,Clothing/clothes,Other/missing
259,רולדין יהודה ה,-23.00,Leisure/Eating out,Other/missing
260,חניון ק.אביב,-11.00,Transport/parking,Other/missing
263,00295 הלוואה - תשלום ריבית (תאריך ערך 02/12),-31.12,Taxes & Insurance/loans,Other/missing


In [98]:
training_data


,desc,amount,cat/tag
0,הפניקס אוטו חובה,-702.00,Basics/Charity
1,WIND MOBILITY GMBH BERLIN DE,-35.50,Transport/scooters
2,"טיב טעם יהודה המכבי ת""א",-110.33,Basics/groceries
3,קיוסק יהודה מכבי 55 תל אב,-32.00,Other/missing
4,רובין הוד,-20.00,Leisure/apps
...,...,...,...
86,תן ביס,-50.80,unsorted/missing
87,"טיב טעם יהודה המכבי ת""א",-21.70,Basics/groceries
88,תן ביס,-40.00,unsorted/missing
89,אריאל ספא אלון ספורט קלאב,-200.00,Clothing/missing
